In [1]:
import re
import copy
import string
import random
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from uralicNLP import uralicApi
from collections import Counter

In [2]:
dt = pd.read_csv("overall_80K.csv")

In [3]:
dt

,fid,target,source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...,...
81141,2345,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,2346,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,2347,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,2348,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [4]:
def delete_quotation_marks(sentence):
        if sentence[0] == '\"' and sentence[-1] == '\"':
            return sentence[1:-1].strip()
        return sentence.strip()

In [5]:
dt['pr_target'] = dt['target'].apply(delete_quotation_marks)
dt['pr_source'] = dt['source'].apply(delete_quotation_marks)

In [6]:
dt

,fid,target,source,pr_target,pr_source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала.","Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит.","Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...,...,...,...
81141,2345,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват...",А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,2346,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,2347,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед...","Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,2348,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [7]:
def find_split_words(dt):
    pattern = r'(?:\b\w\s){3,}\w\b'
    
    def find_mathes(text):
        matches = re.findall(pattern, text)
        return matches
        
    with_split_words = set()
    
    source = dt.pr_source.tolist()
    for i in range(len(source)):
        matches = find_mathes(source[i])
        if len(matches) != 0:
            with_split_words.add(i)

    target = dt.pr_target.tolist()
    for i in range(len(target)):
        matches = find_mathes(target[i])
        if len(matches) != 0:
            with_split_words.add(i)
            
    return with_split_words

In [8]:
id_with_split_words = find_split_words(dt)
print(len(id_with_split_words))

862


In [9]:
def find_strange_symbols(dt):
    xad = '\xad'
    xad_errors = set()
    strange = [ 
        '\uf50f',
        '\uf511',
        '\uf513',
        '\uf518',
        '\uf519',
        '\uf521',
        '\uf523',
        '\uf529',
        '\uf52d'
    ]
    st_i = []
    target = dt.pr_target.tolist()
    for i in range(len(target)):
        for j in range(len(strange)):
            if strange[j] in target[i]:
                st_i.append((i, j))
        if xad in target[i]:
            xad_errors.add(i)

    source = dt.pr_source.tolist()
    for i in range(len(source)):
        for j in range(len(strange)):
            if strange[j] in source[i]:
                st_i.append((i, j))
        if xad in source[i]:
            xad_errors.add(i)
    return xad_errors, st_i

In [10]:
xad_errors, st_i = find_strange_symbols(dt)
len(xad_errors), len(st_i)

(162, 22)

In [11]:
def fix_xad(dt, xad_errors):
    source = dt.pr_source.tolist()
    target = dt.pr_target.tolist()
    for i in xad_errors:
        source[i] = source[i].replace('\xad', '')
        target[i] = target[i].replace('\xad', '')
    dt.pr_source = source
    dt.pr_target = target

In [12]:
fix_xad(dt, xad_errors)
xad_errors, _ = find_strange_symbols(dt)
assert len(xad_errors) == 0

In [13]:
def find_bad_and_swapped_sentences(dt):
    a = ord('а')
    A = ord('А')
    rus = "1234567890! \t#$%()+,./-\"\';:?<>=@[]\\_"
    rus +=  '–' + '—' + '’' + '”' + '…' + '№' + '™' + '\xa0' + '«' + '¬' + '\xad' + '¸' +'»'
    rus += '\uf50f' + '\uf511' + '\uf513' + '\uf518' + '\uf519' + '\uf521' + '\uf523' + '\uf529' + '\uf52d'
    rus += ''.join([chr(i) for i in range(A,A+33)]) + 'Ё'
    rus += ''.join([chr(i) for i in range(a,a+32)] + [chr(a+33)])
    # rus += 'ӣ' + '\'̄\''
    rus += string.ascii_uppercase
    rus += string.ascii_lowercase

    source = dt.pr_source.tolist()
    target = dt.pr_target.tolist()
    letters = set()
    for i in range(len(target)):
        letters = letters | set(target[i])
    mans_letters = letters - set(rus)
    wrong = set()
    swapped = set()
    for i in range(len(source)):
        if len(mans_letters & set(source[i])) > 0 and len(set(target[i]) - set(rus)) == 0 and len(source[i]) > 4 and len(target[i]) > 4:
            swapped.add(i)
        elif len(mans_letters & set(source[i])) > 0 or len(source[i]) <= 4 or len(target[i]) <= 4:
            wrong.add(i)
    return swapped, wrong

In [14]:
swapped, wrong = find_bad_and_swapped_sentences(dt)
len(swapped), len(wrong)

(1462, 236)

In [15]:
def swap_sentences(dt, swapped):
    source = dt.pr_source.tolist()
    target = dt.pr_target.tolist()
    for i in swapped:
        source[i], target[i] = target[i], source[i]
    dt.pr_source = source
    dt.pr_target = target

In [16]:
swap_sentences(dt, swapped)
swapped, _ = find_bad_and_swapped_sentences(dt)
assert len(swapped) == 0

In [21]:
id_to_delete = wrong | set(elem[0] for elem in st_i)

In [22]:
def lemmatize_sentence(sentence):
    def replace_symbols(text):
        for symbol in "!@#$%,.1234567890-/?\'\"\\«»_-:;_…–()”":
        # for symbol in "!@#$%,.1234567890/?\'\"\\«»_:;_…()”":
            text = text.replace(symbol, " ")
        return (' '.join(text.lower().split())).split()
    sentence = replace_symbols(sentence)
    words = []
    for word in sentence:
        norm_form = uralicApi.lemmatize(word, "mns")
        if len(norm_form) == 0:
            words.append(word)
        # elif len(norm_form) > 1:
        #     print(norm_form, word)
        else:
            words.append(sorted(uralicApi.lemmatize(word, "mns"))[0])
    return words

In [23]:
words = []
for sentence in tqdm(dt.pr_target.tolist()):
    words.extend(lemmatize_sentence(sentence))

  0%|          | 0/81146 [00:00<?, ?it/s]

In [24]:
words_count = Counter(words)
len(words_count)

48840

In [25]:
words_to_delete = []
old_words_count = copy.deepcopy(words_count)
words = set(words_count)
for word in tqdm(words):
    if words_count[word] < 10:
        words_to_delete.append(word)
        del words_count[word]
len(words_count)

  0%|          | 0/48840 [00:00<?, ?it/s]

4034

In [26]:
def check_split(dt_check, proportion, delta=10):
    dt_words = []
    zero = []
    l = []
    u = []
    unique = []
    unique_all = []
    for sentence in tqdm(dt_check.pr_target.tolist()):
        dt_words.extend(lemmatize_sentence(sentence))
    dt_words_count = Counter(dt_words)
    dt_words = set(dt_words)
    for word in words_count:
        if words_count[word] * proportion > dt_words_count[word] + delta:
            l.append(word)
        if words_count[word] * proportion < dt_words_count[word] - delta:
            u.append(word)
        if dt_words_count[word] == 0:
            zero.append(word)
    for word in words_to_delete:
        if word in dt_words:
            unique.append(word)
        if dt_words_count[word] == old_words_count[word]:
            unique_all.append(word)
    return l, u, unique, zero, unique_all
        

In [34]:
random.seed(17)
random_shuffled_indexes = dt.index.tolist()
print(len(random_shuffled_indexes))
random_shuffled_indexes = list(set(random_shuffled_indexes) - id_to_delete)
print(len(random_shuffled_indexes))
random.shuffle(random_shuffled_indexes)

train_size = 64892
val_size = 8000
train_indexes = random_shuffled_indexes[:train_size]
val_indexes = random_shuffled_indexes[train_size: train_size + val_size]
test_indexes = random_shuffled_indexes[train_size + val_size:]

81146
80892


In [35]:
l_train, u_train, unique_train, zero_train, unique_all_train = check_split(dt.loc[train_indexes], len(train_indexes) / len(dt))
len(l_train), len(u_train), len(unique_train), len(zero_train), len(unique_all_train)

  0%|          | 0/64892 [00:00<?, ?it/s]

(111, 65, 38257, 1, 31725)

In [36]:
l_val, u_val, unique_val, zero_val, unique_all_val = check_split(dt.loc[val_indexes], len(val_indexes) / len(dt))
len(l_val), len(u_val), len(unique_val), len(zero_val), len(unique_all_val)

  0%|          | 0/8000 [00:00<?, ?it/s]

(42, 51, 7028, 492, 3206)

In [37]:
l_test, u_test, unique_test, zero_test, unique_all_test = check_split(dt.loc[test_indexes], len(test_indexes) / len(dt))
len(l_test), len(u_test), len(unique_test), len(zero_test), len(unique_all_test)

  0%|          | 0/8000 [00:00<?, ?it/s]

(34, 61, 6865, 474, 3019)

In [38]:
dt.loc[train_indexes].to_csv("train.csv")
dt.loc[val_indexes].to_csv("val.csv")
dt.loc[test_indexes].to_csv("test.csv")